## Dynamic time dependent Bayesian Optimization

This example demonstrates time dependent optimization with dynamic parameter changes.

In [1]:
from xopt.generators.bayesian.upper_confidence_bound import TDUpperConfidenceBoundGenerator
from xopt.vocs import VOCS
from xopt.evaluator import Evaluator
import warnings
warnings.filterwarnings("ignore")

In [2]:
# test evaluate function and vocs
import time
start_time = time.time()
#global eval_iteration
eval_iteration = 0
def f(inputs):
    #global eval_iteration
    x_ = inputs["x"]
    current_time = time.time()
    #t_ = eval_iteration
    #eval_iteration += 5
    t_ = current_time - start_time
    y_ = 5*(x_ - t_*1e-2)**2
    return {"y":y_, "time":current_time}

variables = {"x":[-1,1]}
objectives = {"y": "MINIMIZE"}

vocs = VOCS(variables=variables, objectives=objectives)
print(vocs)

def make_evaluator():
    return Evaluator(function=f)
evaluator = make_evaluator()

def make_generator():
    generator = TDUpperConfidenceBoundGenerator(vocs)
    generator.options.n_initial = 3
    generator.options.acq.added_time=2.0
    generator.options.acq.beta = 2.0
    #generator.options.model.use_low_noise_prior = True
    return generator

generator = make_generator()

variables={'x': [-1.0, 1.0]} constraints={} objectives={'y': 'MINIMIZE'} constants={} linked_variables={}


In [3]:
from xopt import Xopt
X = Xopt(evaluator=make_evaluator(), generator=make_generator(), vocs=vocs)

In [4]:
# See current options
X.generator.options.dict()

{'optim': {'num_restarts': 5,
  'raw_samples': 20,
  'sequential': True,
  'use_nearby_initial_points': True,
  'max_travel_distances': None},
 'acq': {'proximal_lengthscales': None,
  'use_transformed_proximal_weights': True,
  'monte_carlo_samples': 128,
  'added_time': 2.0,
  'beta': 2.0},
 'model': {'function': <function xopt.generators.bayesian.models.time_dependent.create_time_dependent_model(data, vocs, added_time: float = 0.0, use_conservative_prior_lengthscale: bool = False, use_conservative_prior_mean: bool = False, use_low_noise_prior: bool = False) -> botorch.models.model_list_gp_regression.ModelListGP>,
  'kwargs': {'use_conservative_prior_lengthscale': False,
   'use_conservative_prior_mean': False,
   'use_low_noise_prior': False}},
 'n_initial': 3}

In [5]:
modified_options = {'acq':{'added_time':3.5}}

In [6]:
print(X.generator.generate(1))

{'x': array([-0.96877611,  0.44915158, -0.14532793])}


In [7]:
# this does not create any permanent changes
print(X.generator.generate_custom(1, config_changes=modified_options, is_permanent=False))
print(X.generator.options.dict()['acq'])

({'x': array([0.65207923, 0.32631132, 0.27007975])}, TDUCBOptions(optim=OptimOptions(num_restarts=5, raw_samples=20, sequential=True, use_nearby_initial_points=True, max_travel_distances=None), acq=TDUpperConfidenceBoundOptions(proximal_lengthscales=None, use_transformed_proximal_weights=True, monte_carlo_samples=128, added_time=3.5, beta=2.0), model=TDModelOptions(function=<function create_time_dependent_model at 0x0000022E33F29670>, kwargs=kwargs(use_conservative_prior_lengthscale=False, use_conservative_prior_mean=False, use_low_noise_prior=False)), n_initial=3))
{'proximal_lengthscales': None, 'use_transformed_proximal_weights': True, 'monte_carlo_samples': 128, 'added_time': 2.0, 'beta': 2.0}


In [8]:
# Modifications are validated against model and can be refused
try:
    X.generator.generate_custom(1, config_changes={'acq':{'added_time':'abc'}})
except Exception as ex:
    print(ex)

1 validation error for TDUCBOptions
acq -> added_time
  value is not a valid float (type=type_error.float)


In [9]:
# Now, changes are retained
print(X.generator.generate_custom(1, config_changes=modified_options, is_permanent=True))
print(X.generator.options.dict()['acq'])

({'x': array([ 0.86362214, -0.1606999 ,  0.29459456])}, TDUCBOptions(optim=OptimOptions(num_restarts=5, raw_samples=20, sequential=True, use_nearby_initial_points=True, max_travel_distances=None), acq=TDUpperConfidenceBoundOptions(proximal_lengthscales=None, use_transformed_proximal_weights=True, monte_carlo_samples=128, added_time=3.5, beta=2.0), model=TDModelOptions(function=<function create_time_dependent_model at 0x0000022E33F29670>, kwargs=kwargs(use_conservative_prior_lengthscale=False, use_conservative_prior_mean=False, use_low_noise_prior=False)), n_initial=3))
{'proximal_lengthscales': None, 'use_transformed_proximal_weights': True, 'monte_carlo_samples': 128, 'added_time': 3.5, 'beta': 2.0}


In [10]:
# If step interface is used, any options changes are stored in history
X = Xopt(evaluator=make_evaluator(), generator=make_generator(), vocs=vocs)
X.step()
X.step(generator_opts=modified_options, is_permanent=False)
X.step()

In [11]:
for x in X._generator_options_history:
    print(x)
    print()

{'is_permanent': False, 'config_change': None, 'n': 1, 'effective_config': {'optim': {'num_restarts': 5, 'raw_samples': 20, 'sequential': True, 'use_nearby_initial_points': True, 'max_travel_distances': None}, 'acq': {'proximal_lengthscales': None, 'use_transformed_proximal_weights': True, 'monte_carlo_samples': 128, 'added_time': 2.0, 'beta': 2.0}, 'model': {'function': <function create_time_dependent_model at 0x0000022E33F29670>, 'kwargs': {'use_conservative_prior_lengthscale': False, 'use_conservative_prior_mean': False, 'use_low_noise_prior': False}}, 'n_initial': 3}}

{'is_permanent': False, 'config_change': {'acq': {'added_time': 3.5}}, 'n': 1, 'effective_config': TDUCBOptions(optim=OptimOptions(num_restarts=5, raw_samples=20, sequential=True, use_nearby_initial_points=True, max_travel_distances=None), acq=TDUpperConfidenceBoundOptions(proximal_lengthscales=None, use_transformed_proximal_weights=True, monte_carlo_samples=128, added_time=3.5, beta=2.0), model=TDModelOptions(functi

In [12]:
# History can be used to recover state at a specific step
X.rollback(step=1)
for x in X._generator_options_history:
    print(x)
    print()
display(X.data)

{'is_permanent': False, 'config_change': None, 'n': 1, 'effective_config': {'optim': {'num_restarts': 5, 'raw_samples': 20, 'sequential': True, 'use_nearby_initial_points': True, 'max_travel_distances': None}, 'acq': {'proximal_lengthscales': None, 'use_transformed_proximal_weights': True, 'monte_carlo_samples': 128, 'added_time': 2.0, 'beta': 2.0}, 'model': {'function': <function create_time_dependent_model at 0x0000022E33F29670>, 'kwargs': {'use_conservative_prior_lengthscale': False, 'use_conservative_prior_mean': False, 'use_low_noise_prior': False}}, 'n_initial': 3}}



,x,y,time,xopt_runtime,xopt_error
1,0.763794,2.126092,1.667333e+09,0.000005,False


In [ ]:
# Full loop with adaptive time step (for example, because of thermal effects each next measurement is slower)
X = Xopt(evaluator=make_evaluator(), generator=make_generator(), vocs=vocs)
eval_iteration = 0
for i in range(20):
    X.step(generator_opts={'acq':{'added_time':1.0*(i/5+1)}})
    time.sleep(0.1)

In [ ]:
X.data

In [ ]:
# plot model
import torch
from matplotlib import pyplot as plt  # plot model predictions
data = X.data

xbounds = generator.vocs.bounds
tbounds = [data["time"].min(), data["time"].max()]

def gt(inpts):
    return 5*(inpts[:,1] - (inpts[:,0] - start_time)*1e-2)**2

model = X.generator.model
n = 200
t = torch.linspace(*tbounds, n, dtype=torch.double)
x = torch.linspace(*xbounds.flatten(), n, dtype=torch.double)
tt, xx = torch.meshgrid(t, x)
pts = torch.hstack([ele.reshape(-1, 1) for ele in (tt, xx)]).double()

#NOTE: the model inputs are such that t is the last dimension
gp_pts = torch.flip(pts, dims=[-1])

gt_vals = gt(pts)

with torch.no_grad():
    post = model.posterior(gp_pts)

    mean = post.mean
    std = torch.sqrt(post.variance)

    fig, ax = plt.subplots()
    ax.set_title("model mean")
    ax.set_xlabel("unix time")
    ax.set_ylabel("x")
    c = ax.pcolor(tt, xx, mean.reshape(n,n))
    fig.colorbar(c)

    fig2, ax2 = plt.subplots()
    ax2.set_title("model uncertainty")
    ax2.set_xlabel("unix time")
    ax2.set_ylabel("x")
    c = ax2.pcolor(tt, xx, std.reshape(n,n))
    fig2.colorbar(c)

    ax.plot(data["time"].to_numpy(), data["x"].to_numpy(),"oC1")
    ax2.plot(data["time"].to_numpy(), data["x"].to_numpy(),"oC1")

    fig3, ax3 = plt.subplots()
    ax3.set_title("ground truth value")
    ax3.set_xlabel("unix time")
    ax3.set_ylabel("x")
    c = ax3.pcolor(tt, xx, gt_vals.reshape(n,n))
    fig3.colorbar(c)

In [ ]:
list(model.named_parameters())